In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode, lower, regexp_replace, collect_set, size, array_intersect
import socket
import time

# --- Configuração do Spark Session ---
local_ip = socket.gethostbyname(socket.gethostname())

spark = SparkSession.builder \
    .appName("Workload Pesado - Vocabulario Comum Entre Generos") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

print("Spark Session iniciada com sucesso!")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/09 20:25:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session iniciada com sucesso!


In [2]:
def executar_workload_intersecao_artistas_generos_diferentes():
    """
    Executa um workload pesado: encontra o número de palavras em comum
    entre artistas, inclusive de gêneros diferentes.
    Força milhares de combinações para estressar o cluster.
    """
    from pyspark.sql import functions as F
    from pyspark.sql.window import Window
    import time

    inicio = time.time()

    # --- 1. Carregar e Preparar os Dados ---
    caminho_parquet = "/spark-data/musicas_limpas_cluster.parquet"
    df = spark.read.parquet(caminho_parquet)

    # NÃO filtra os gêneros para forçar maior carga
    df_palavras = df.withColumn("palavra", F.explode(F.split(F.lower(F.regexp_replace("text", r'[\W_]+', ' ')), ' '))) \
                    .select("main_genre", "Artist(s)", "palavra") \
                    .filter(F.col("palavra") != "")

    # Limita para os top 2000 artistas com mais músicas no total
    df_top_artistas = df.groupBy("Artist(s)").count()
    df_top_artistas = df.groupBy("Artist(s)").count() \
                    .orderBy(F.col("count").desc()) \
                    .limit(2000) \
                    .select("Artist(s)")

    # Filtra df_palavras para manter só esses artistas
    df_palavras_filtrado = df_palavras.join(df_top_artistas, on="Artist(s)", how="inner")

    # Gera vocabulário de cada artista
    df_vocabulario = df_palavras_filtrado.groupBy("Artist(s)", "main_genre") \
                                         .agg(F.collect_set("palavra").alias("vocabulario"))

    # Cache para otimizar
    df_vocabulario = df_vocabulario.repartition(200).cache()
    print(f"Total de artistas com vocabulário: {df_vocabulario.count()}")

    # --- 2. Self Join completo (entre gêneros diferentes também) ---
    df_a1 = df_vocabulario.alias("a1")
    df_a2 = df_vocabulario.alias("a2")

    df_pares = df_a1.join(
        df_a2,
        (F.col("a1.`Artist(s)`") < F.col("a2.`Artist(s)`")) &
        (F.col("a1.main_genre") != F.col("a2.main_genre")),
        "inner"
    )

    print(f"Número de pares comparados: {df_pares.count()}")

    # --- 3. Cálculo da Interseção ---
    df_resultado = df_pares.withColumn(
        "palavras_em_comum",
        F.size(F.array_intersect(F.col("a1.vocabulario"), F.col("a2.vocabulario")))
    ).select(
        F.col("a1.`Artist(s)`").alias("artista_1"),
        F.col("a1.main_genre").alias("genero_1"),
        F.col("a2.`Artist(s)`").alias("artista_2"),
        F.col("a2.main_genre").alias("genero_2"),
        "palavras_em_comum"
    ).orderBy(F.col("palavras_em_comum").desc())

    fim = time.time()
    duracao = fim - inicio
    throughput = 424/duracao

    print(f"\n--- Análise de Desempenho ---")
    print(f"Tempo total de execução: {duracao:.2f} segundos")
    print(f"Throughput da execução: {throughput:.2f} MB/s")
    print(f"----------------------------")
    


executar_workload_intersecao_artistas_generos_diferentes()

spark.stop()

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/spark-data/musicas_limpas.parquet.